In this example, we will explain why a certain sentence is classified by a logistic regression as having negative or positive sentiment.

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import spacy
from alibi.explainers import AnchorText
from alibi.datasets import movie_sentiment

### Load movie review dataset

The movie review dataset can be found here: https://www.kaggle.com/nltkdata/sentence-polarity#sentence_polarity.zip.
You can extract the files in a folder of your choosing (the notebook assumes *'sentence_polarity'* in the current path). 

In [ ]:
data, labels = movie_sentiment(path='sentence_polarity')

Define shuffled training and test set

In [ ]:
train, test, train_labels, test_labels = train_test_split(data, labels, test_size=.2, random_state=0)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

### Apply CountVectorizer to training set

In [ ]:
vectorizer = CountVectorizer(min_df=1)
vectorizer.fit(train)

### Fit model

In [ ]:
clf = LogisticRegression()
clf.fit(vectorizer.transform(train), train_labels)

### Define prediction function

In [ ]:
def predict_fn(texts):
    return clf.predict(vectorizer.transform(texts))

### Make predictions on train and test sets

In [ ]:
preds_train = predict_fn(train)
preds_test = predict_fn(test)
print('Train accuracy', accuracy_score(train_labels, preds_train))
print('Test accuracy', accuracy_score(test_labels, preds_test))

### Load spaCy model

English multi-task CNN trained on OntoNotes, with GloVe vectors trained on Common Crawl. Assigns word vectors, context-specific token vectors, POS tags, dependency parse and named entities.

Note: you must have spacy installed. Run:

        pip install spacy && python -m spacy download en_core_web_lg

In [ ]:
nlp = spacy.load('en_core_web_lg')

### Initialize anchor text explainer

In [ ]:
explainer = AnchorText(nlp, predict_fn)

### Explain a prediction

In [ ]:
class_names = ['negative', 'positive']

Prediction:

In [ ]:
np.random.seed(0)
text = 'This is a good book .'
pred = class_names[predict_fn([text])[0]]
alternative =  class_names[1 - predict_fn([text])[0]]
print('Prediction: %s' % pred)

Explanation:

In [ ]:
explanation = explainer.explain(text, threshold=0.95, use_proba=False, use_unk=True)

use_unk=True means we will perturb examples by replacing words with UNKs. Let us now take a look at the anchor. The word 'good' basically guarantees a positive prediction. This is because the UNKs do not take instances like 'not good' into account.

In [ ]:
print('Anchor: %s' % (' AND '.join(explanation['names'])))
print('Precision: %.2f' % explanation['precision'])
print('\nExamples where anchor applies and model predicts %s:' % pred)
print('\n'.join([x[0] for x in explanation['raw']['examples'][0]['covered_true']]))
print('\nExamples where anchor applies and model predicts %s:' % alternative)
print('\n'.join([x[0] for x in explanation['raw']['examples'][0]['covered_false']]))

### Changing the perturbation distribution
Let's try this with another perturbation distribution, namely one that replaces words by similar words instead of UNKs.

Explanation:

In [ ]:
explanation = explainer.explain(text, threshold=0.95, use_proba=True, use_unk=False)

The anchor now shows that we need more to guarantee the positive prediction:

In [ ]:
print('Anchor: %s' % (' AND '.join(explanation['names'])))
print('Precision: %.2f' % explanation['precision'])
print('\nExamples where anchor applies and model predicts %s:' % pred)
print('\n'.join([x[0] for x in explanation['raw']['examples'][0]['covered_true']]))
print('\nExamples where anchor applies and model predicts %s:' % alternative)
print('\n'.join([x[0] for x in explanation['raw']['examples'][0]['covered_false']]))